In [ ]:
import numpy as np
import timeit
import torch

from context import rf_pool

In [ ]:
# init rfs and data
img_shape = [20,20]
mu, sigma = rf_pool.utils.lattice.init_uniform_lattice(np.array(img_shape)//2, (4,4), 20, sigma_init=2.)
rfs = rf_pool.utils.lattice.mask_kernel_lattice(mu, sigma, img_shape)

a = torch.as_tensor(np.random.rand(10,64,1,*img_shape), dtype=torch.float32)
b = rfs.clone();

In [ ]:
# get rf_pool output and timing
t1 = []
a1 = a.numpy()
b1 = b.numpy()
for _ in range(1000):
    s = timeit.default_timer()
    out1 = rf_pool.pool.max_pool(a1.reshape((-1,*img_shape)), b1).reshape(a.shape)
    e = timeit.default_timer()
    t1.append(e - s)
print('RF-Pool takes %0.4f secs' % np.mean(t1))

In [ ]:
# compare to previous rf_pool approach
t0 = []
for _ in range(100):
    s = timeit.default_timer()
    h_mean = torch.mul(a, b)
    h_sample = torch.reshape(rf_pool.ops.max_index(h_mean.flatten(-2)), h_mean.shape)
    p_mean = torch.mul(h_mean, h_sample)
    out0 = torch.max(p_mean, -3)[0]
    e = timeit.default_timer()
    t0.append(e - s)
print('Previous approach takes %0.4f secs' % np.mean(t0))

In [ ]:
print('RF-Pool is %0.2f times faster than previous approach' % (np.mean(t0) / np.mean(t1)))

In [ ]:
print('RF-Pool output == previous output: %a' % (np.allclose(out0.numpy(), out1.reshape(out0.shape))))

In [ ]:
# compare to 2x2 maxpool
t_maxpool = []
for _ in range(1000):
    s = timeit.default_timer()
    torch.nn.functional.max_pool2d(a.squeeze(2), (2,2))
    e = timeit.default_timer()
    t_maxpool.append(e - s)
print('2x2 MaxPool takes %0.4f secs' % np.mean(t_maxpool))

In [ ]:
print('RF-Pool is %0.2f times faster than 2x2 MaxPool' % (np.mean(t_maxpool) / np.mean(t1)))

In [ ]:
def test_fn(a, b):
    return np.mean(a) - np.mean(b)

# bootstrap test difference
diff0 = np.mean(t0) - np.mean(t1)

z_mean = np.mean(t0 + t1)
x_prime = np.array(t1) - np.mean(t1) + z_mean
y_prime = np.array(t0) - np.mean(t0) + z_mean

diffs = rf_pool.utils.functions.bootstrap(x_prime, y_prime, n_samples=1000, fn=test_fn)

print('RF-Pool faster than previous approach: p = %a' % np.mean(diffs > diff0))